<a href="https://colab.research.google.com/github/ayushipradhan30/AIML-mini-projects/blob/main/Neural_Network_and_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Activation, Dense, BatchNormalization, Dropout
from keras import optimizers
print('TensorFlow Version: ', tf.__version__)

TensorFlow Version:  2.17.0


In [7]:
hdf = h5py.File('SVHN_single_grey1.h5','r')
hdf.keys()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'SVHN_single_grey1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
x_train = hdf['X_train']
x_test = hdf['X_test']
x_val = hdf['X_val']
y_train = hdf['y_train']
y_test = hdf['y_test']
y_val = hdf['y_val']

In [ ]:
print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)